In [1]:
import os, sys

os.environ["HF_DATASETS_OFFLINE"] = "0"
os.environ["HF_DATASETS_CACHE"] = "/tmp"     # points to temp dir
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["HF_DATASETS_DISABLE_CACHING"] = "1"  # 👈 full off switch

project_root = "/home/michael/workspace/phrasely"
sys.path.insert(0, os.path.join(project_root, "src"))
os.chdir(project_root)


import random
import warnings
from tqdm import TqdmWarning, tqdm
import pandas as pd
from phrasely.pipeline import run_pipeline
from phrasely.data_loading.cc100_loader import CC100Loader
from phrasely.embeddings.phrase_embedder import PhraseEmbedder
from phrasely.data_loading.cc100_offline_loader import CC100OfflineLoader
import logging

logging.getLogger().setLevel(logging.INFO)



warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=TqdmWarning)

/home/michael/micromamba/envs/phrasely-gpu/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def setup_logger(name: str) -> logging.Logger:
    logger = logging.getLogger(name)
    if not logger.handlers:
        handler = logging.StreamHandler(sys.stdout)
        formatter = logging.Formatter("%(message)s")
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
    return logger

In [3]:
setup_logger('phrasely')

<Logger phrasely (INFO)>

In [4]:
from phrasely.pipeline import run_pipeline
from phrasely.data_loading.cc100_offline_loader import CC100OfflineLoader

# --- keep this small just to verify everything completes cleanly ---
loader_args = {
    "arrow_dir": "data_cache/cc100",
    "max_phrases": 100_000,
    "chunk_size": 50_000,
}

result = run_pipeline(
    CC100OfflineLoader,
    loader_kwargs=loader_args,
    n_components=100,
    use_gpu=True,
)

# save the results for later exploration
result.save("data_cache/run_cc100_100k")

print("\n✅ End-to-end pipeline finished successfully!")
result.summary()

🚀 Starting Phrasely pipeline...
Detected GPU VRAM: 3.8 GB
Adaptive GPU limits — SVD: 190,000 rows, HDBSCAN: 190,000 rows.
▶️  Loading phrases...
Resuming from 20 existing chunks in data_cache/cc100_phrases_en_parts


Filtering CC100 shards: 1000000it [00:00, 25420024242.42it/s]                                       

Filtered and saved 1,000,000 phrases.
Merging parts from data_cache/cc100_phrases_en_parts ...


Merged 1,500,000 phrases into data_cache/cc100_phrases_en.parquet
Loaded 1,500,000 CC100 offline phrases.
Loading phrases completed in 1.377s.
▶️  Embedding phrases...
PhraseEmbedder using model=paraphrase-MiniLM-L6-v2, device=cuda, VRAM≈3.8 GB, batch_size=8
Converted model to fp16 for reduced VRAM usage.
Loading cached embeddings from data_cache/embeddings_paraphrase-MiniLM-L6-v2_4f64f06e08fffd13fd3698dd29470c07.npy
🧹 Freed GPU memory after embedding.
Embedding phrases completed in 1.122s.
▶️  Reducing dimensions...
SVDReducer: using GPU backend for TruncatedSVD.
SVDReducer: reduced 384 → 100dimensions.
🧹 Freed GPU memory after SVD reduction.
Reducing dimensions completed in 0.488s.
▶️  Clustering phrases...
HDBSCANClusterer: using GPU backend.
HDBSCANClusterer: found 41 clusters (+ noise).
🧹 Freed GPU memory after clustering.
Clustering phrases completed in 14.006s.
▶️  Selecting medoids...


TypeError: MedoidSelector.select() missing 1 required positional argument: 'labels'